In [1]:
import numpy as np

In [2]:
import MeCab as mc
import mojimoji
m = mc.Tagger("-Ochasen")

In [3]:
def mecab_tokenizer(text: str):
    """
    テキストを分かち書きするメソッド
    :param text: 分割したいテキスト
    :return: 分割後のテキスト
    """

    node = m.parseToNode(text)
    word_list = list()
    while node:
        if node.surface != "":
            res = node.feature.split(",")
            word_type = res[0]
            if word_type in ['名詞', "動詞", "形容詞", "副詞"]:  # 名詞, 動詞, 形容詞, 副詞のみを抽出
                basic_word = res[6]
                if basic_word != "*":
                    word_list.append(basic_word)
                else:
                    word_list.append('[UNK]')  # 未知語の場合は[UNK]トークンに置き換え
        node = node.next
        if node is None:
            break
    return word_list

In [4]:
def clean_text(text: str):
    """
    テキストの正規化
    :param text: 正規化するテキスト
    :return: 正規化後のテキスト
    """
    text = mojimoji.han_to_zen(text, digit=False, ascii=False)  # 半角文字を全角文字に統一(数字, 英語以外)
    text = mojimoji.zen_to_han(text, kana=False)  # 全角文字を半角文字に統一(かな以外)
    text = text.lower()  # 小文字に統一
    return text

In [5]:
from gensim.models import KeyedVectors

In [6]:
def vectorize_word(text: str):
    """
    元word_analysis()
    ベクトル化を行うメソッド
    :param text: ベクトル化するテキスト
    :return: ベクトル化したテキスト(array型)
    """

    text = clean_text(text)

    V = list()  # 文章のベクトル(200次元)を格納

    # 文章の対して, 文章中の単語のベクトルの平均を求める処理を行う
    word_list = mecab_tokenizer(text)
    v = np.array([0.0] * 200)
    word_num = 0
    for word in word_list:
        try:
            v += np.array(model[word])
        except KeyError as error:
            continue
        except ValueError as verror:
            continue
        word_num += 1

    try:
        v = v / word_num
    except ZeroDivisionError as e:
        print(f'ZeroDivisionError: {e}')

    return v

def cos_sim(v1, v2):
    """
    cos類似度を計算
    :param v1: word2vecによりベクトル化したテキスト1(ndarray型)
    :param v2: word2vecによりベクトル化したテキスト2(ndarray型)
    :return: 二つのテキストのcos類似度
    """
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [7]:
import pickle

In [8]:
with open('word2vec_model.pickle', mode='rb') as f:
    model = pickle.load(f)

In [10]:
with open('C:/Users/kenbun/PycharmProjects/ProtKenbunTalk/Excel_News/headline_list.pickle', mode='rb') as f:
    healdline_list = pickle.load(f)

In [11]:
# test_keyword = '大分'
test_keyword = '縣'

In [12]:
def get_all_similarity(headline_list, speech_text: str):
    """
    全ての見出しと発言の類似度分析を行い, 類似度で降順にソートして返す
    :param headline_list: 見出しをまとめたリスト<br> ひとつの要素に(ファイルパス, 番号, (見出し, ベクトル)) -> の形で格納している
    :param speech_text: 発言
    :return: 類似度で降順にソートしたリスト [((会社名, 出版年, 出版付き, 番号, 見出し), 類似度)] -> この形になっている
    """
    # speech_text = '物価下がらなかったのね'
    speech_v = vectorize_word(speech_text)  # 発言をベクトル化

    similarity_dic = {}

    for company, year, month, day, headline, vector in headline_list:
        cs = cos_sim(speech_v, vector)  # 発言と見出しテキストのcos類似度を計算

        if cs >= 0.75:  # cos類似度が閾値以上であれば提示する見出しリストに加える
            similarity_dic[(company, year, month, day, headline)] = cs

    return sorted(similarity_dic.items(), key=lambda x: x[1], reverse=True)

In [13]:
result = get_all_similarity(healdline_list, test_keyword)

c:\Users\kenbun\anaconda3\envs\nlp_bot\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide


In [14]:
result

[]